In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline
import sklearn
from sklearn import linear_model
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

### Directions

Engineer your features, then create three models. Each model will be run on a training set and a test-set (or multiple test-sets, if you take a folds approach). The models should be:

+ Vanilla logistic regression
+ Ridge logistic regression
+ Lasso logistic regression

If you're stuck on how to begin combining your two new modeling skills, here's a hint: the SKlearn LogisticRegression method has a "penalty" argument that takes either 'l1' or 'l2' as a value.

In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df.shape[0]

1460

In [5]:
trainsize = int(df.shape[0]/2)

In [6]:
trainsize

730

In [7]:
df.select_dtypes(include=["object"]).head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [8]:
for col in df.select_dtypes(include=["object"]):
    print(col)
    print(df[col].nunique())

MSZoning
5
Street
2
Alley
2
LotShape
4
LandContour
4
Utilities
2
LotConfig
5
LandSlope
3
Neighborhood
25
Condition1
9
Condition2
8
BldgType
5
HouseStyle
8
RoofStyle
6
RoofMatl
8
Exterior1st
15
Exterior2nd
16
MasVnrType
4
ExterQual
4
ExterCond
5
Foundation
6
BsmtQual
4
BsmtCond
4
BsmtExposure
4
BsmtFinType1
6
BsmtFinType2
6
Heating
6
HeatingQC
5
CentralAir
2
Electrical
5
KitchenQual
4
Functional
7
FireplaceQu
5
GarageType
6
GarageFinish
3
GarageQual
5
GarageCond
5
PavedDrive
3
PoolQC
3
Fence
4
MiscFeature
4
SaleType
9
SaleCondition
6


In [9]:
df.select_dtypes(exclude=["object"]).columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [10]:
df["YearBuilt"].unique()

array([2003, 1976, 2001, 1915, 2000, 1993, 2004, 1973, 1931, 1939, 1965,
       2005, 1962, 2006, 1960, 1929, 1970, 1967, 1958, 1930, 2002, 1968,
       2007, 1951, 1957, 1927, 1920, 1966, 1959, 1994, 1954, 1953, 1955,
       1983, 1975, 1997, 1934, 1963, 1981, 1964, 1999, 1972, 1921, 1945,
       1982, 1998, 1956, 1948, 1910, 1995, 1991, 2009, 1950, 1961, 1977,
       1985, 1979, 1885, 1919, 1990, 1969, 1935, 1988, 1971, 1952, 1936,
       1923, 1924, 1984, 1926, 1940, 1941, 1987, 1986, 2008, 1908, 1892,
       1916, 1932, 1918, 1912, 1947, 1925, 1900, 1980, 1989, 1992, 1949,
       1880, 1928, 1978, 1922, 1996, 2010, 1946, 1913, 1937, 1942, 1938,
       1974, 1893, 1914, 1906, 1890, 1898, 1904, 1882, 1875, 1911, 1917,
       1872, 1905], dtype=int64)

In [11]:
df["BuildOldHouse"] = np.where(df["YearBuilt"] < 1930, 1, 0)

In [12]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,BuildOldHouse
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,0


In [13]:
df["BuildSemiOldHouse"] = np.where((df["YearBuilt"] > 1930) & (df["YearBuilt"] < 1965), 1, 0)

In [14]:
df["BuiltNotOldHouse"] = np.where((df["YearBuilt"] > 1965) & (df["YearBuilt"] < 1990), 1, 0)

In [15]:
df["BuildContemporaryHouse"] = np.where(df["YearBuilt"] > 1990, 1, 0)

In [16]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,BuildOldHouse,BuildSemiOldHouse,BuiltNotOldHouse,BuildContemporaryHouse
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,2,2008,WD,Normal,208500,0,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,5,2007,WD,Normal,181500,0,0,1,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,9,2008,WD,Normal,223500,0,0,0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,2,2006,WD,Abnorml,140000,1,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,12,2008,WD,Normal,250000,0,0,0,1


In [17]:
df["YrSold"].unique()

array([2008, 2007, 2006, 2009, 2010], dtype=int64)

In [18]:
df = pd.get_dummies(df)

In [19]:
df.shape

(1460, 294)

In [20]:
for col in df.columns:
    print(col)
    print(df[col].isna().sum())

Id
0
MSSubClass
0
LotFrontage
259
LotArea
0
OverallQual
0
OverallCond
0
YearBuilt
0
YearRemodAdd
0
MasVnrArea
8
BsmtFinSF1
0
BsmtFinSF2
0
BsmtUnfSF
0
TotalBsmtSF
0
1stFlrSF
0
2ndFlrSF
0
LowQualFinSF
0
GrLivArea
0
BsmtFullBath
0
BsmtHalfBath
0
FullBath
0
HalfBath
0
BedroomAbvGr
0
KitchenAbvGr
0
TotRmsAbvGrd
0
Fireplaces
0
GarageYrBlt
81
GarageCars
0
GarageArea
0
WoodDeckSF
0
OpenPorchSF
0
EnclosedPorch
0
3SsnPorch
0
ScreenPorch
0
PoolArea
0
MiscVal
0
MoSold
0
YrSold
0
SalePrice
0
BuildOldHouse
0
BuildSemiOldHouse
0
BuiltNotOldHouse
0
BuildContemporaryHouse
0
MSZoning_C (all)
0
MSZoning_FV
0
MSZoning_RH
0
MSZoning_RL
0
MSZoning_RM
0
Street_Grvl
0
Street_Pave
0
Alley_Grvl
0
Alley_Pave
0
LotShape_IR1
0
LotShape_IR2
0
LotShape_IR3
0
LotShape_Reg
0
LandContour_Bnk
0
LandContour_HLS
0
LandContour_Low
0
LandContour_Lvl
0
Utilities_AllPub
0
Utilities_NoSeWa
0
LotConfig_Corner
0
LotConfig_CulDSac
0
LotConfig_FR2
0
LotConfig_FR3
0
LotConfig_Inside
0
LandSlope_Gtl
0
LandSlope_Mod
0
LandSlope_Sev
0

In [21]:
df = df.dropna()

df.shape

(1121, 294)

In [22]:
df["Affordable"] = np.where(df["SalePrice"] < 165000, 1, 0)

In [23]:
df["SalePrice"].describe()

count      1121.000000
mean     185506.152542
std       82999.159004
min       35311.000000
25%      131000.000000
50%      164900.000000
75%      219500.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [24]:
del df["SalePrice"]

In [25]:
df["Affordable"][10:30]

11    0
13    0
15    1
17    1
18    1
19    1
20    0
21    1
22    0
23    1
25    0
26    1
27    0
28    0
29    1
30    1
32    0
33    0
34    0
35    0
Name: Affordable, dtype: int32

In [26]:
df_test = df.iloc[trainsize:, :-1]
df_train = df.iloc[:trainsize, :-1]
outcome_test = df["Affordable"][trainsize:].ravel()
outcome_train = df["Affordable"][:trainsize].ravel()

### Logistic Regression

In [27]:
lr = LogisticRegression(C=1e9)
X = df_train
y = outcome_train

model_1 = lr.fit(X,y)

In [28]:
print("coefficients for vanilla logistic regression:")
print(model_1.coef_)
print(model_1.intercept_)


coefficients for vanilla logistic regression:
[[-7.37736001e-05  1.60548362e-02 -2.03297465e-02 -7.75150377e-05
  -1.11194021e+00 -4.65853138e-01 -4.16628860e-02  8.11861749e-04
  -2.90200706e-03 -1.19962656e-03 -1.21450899e-03 -1.51758568e-04
  -2.56589412e-03 -8.74802472e-04 -8.86767685e-04 -3.16990201e-03
  -4.93147217e-03  1.06289243e-01 -2.59849526e-01 -8.30708740e-01
  -3.73143159e-01 -7.63804855e-02  2.90168398e-01  5.04588937e-02
  -1.06572180e-01 -4.12366552e-03 -1.71618220e-01 -1.69309308e-03
  -4.66584265e-04 -6.14228363e-03 -2.01756650e-03 -2.86823334e-03
  -7.80010715e-03 -1.68969341e-02  4.89320672e-03  4.42360683e-04
   5.76823760e-02 -2.76537082e-01  1.02353610e-01  1.81943594e-01
  -1.20712502e-01  4.78051648e-02 -2.27807892e-01  3.91070639e-02
  -5.36437253e-01  6.77856394e-01  2.47869026e-04  2.75607955e-04
   5.95627400e-02 -1.51170707e-01 -9.08077745e-02 -2.70652812e-03
  -2.37060989e-06  9.40401502e-02  1.74754958e-01 -7.67666955e-03
   1.87702791e-01 -3.54257602e

In [29]:
model_1_pred_y = model_1.predict(X)

In [30]:
print("Number of mislabeled points out of a total of {} points: {}".format(df_train.shape[0], (outcome_train != model_1_pred_y).sum()))

Number of mislabeled points out of a total of 730 points: 23


In [31]:
print("Accuracy of vanilla logistic regression on training set: ", 
      ((outcome_train == model_1_pred_y).sum()) / df_train.shape[0])

Accuracy of vanilla logistic regression on training set:  0.9684931506849315


In [32]:
lr = LogisticRegression(C=1e9)
X = df_test
y = outcome_test

y_pred = model_1.predict(X)


In [33]:
print("Accuracy of vanilla logistic regression on test set: ", 
      ((outcome_test == y_pred).sum()) / df_test.shape[0])

Accuracy of vanilla logistic regression on test set:  0.9232736572890026


### Ridge Regression

In [34]:
lambdas = [.02, .05, .10, .15, .25, .35, .5, .55, .65, .75, .95, 1, 1.1]

for lam in lambdas:
    ridgereg = LogisticRegression(C=1/lam, fit_intercept=False, penalty="l2")
    ridgereg.fit(df_train, outcome_train)
    print(lam, ridgereg.score(df_train, outcome_train))

0.02 0.9876712328767123
0.05 0.9821917808219178
0.1 0.9876712328767123
0.15 0.9876712328767123
0.25 0.9904109589041096
0.35 0.989041095890411
0.5 0.9753424657534246
0.55 0.9821917808219178
0.65 0.9712328767123287
0.75 0.9821917808219178
0.95 0.963013698630137
1 0.963013698630137
1.1 0.9821917808219178


In [35]:
ridgereg = LogisticRegression(C=1/.25, fit_intercept=False, penalty="l2")
ridgereg.fit(df_train, outcome_train)
print("Accuracy of ridge regression on training set: ", 
      ridgereg.score(df_train, outcome_train))

Accuracy of ridge regression on training set:  0.9904109589041096


In [36]:
y_pred = ridgereg.predict(df_test)
print("Accuracy of ridge regression on test set: ", 
      ((outcome_test == y_pred).sum()) / df_test.shape[0])

Accuracy of ridge regression on test set:  0.9130434782608695


### Lasso Regression

In [37]:
lambdas = [.02, .05, .10, .15, .25, .35, .5, .55, .65, .75, .95, 1, 1.1]


for lam in lambdas:
    lass = LogisticRegression(C=1/lam, fit_intercept=False, penalty="l1")
    lass.fit(df_train, outcome_train)
    print(lam, lass.score(df_train, outcome_train))

0.02 1.0
0.05 1.0
0.1 1.0
0.15 1.0
0.25 0.9986301369863013
0.35 0.989041095890411
0.5 0.9863013698630136
0.55 0.9849315068493151
0.65 0.9835616438356164
0.75 0.9821917808219178
0.95 0.9780821917808219
1 0.9780821917808219
1.1 0.9753424657534246


In [46]:
lass = LogisticRegression(C=1/.02, fit_intercept=False, penalty="l1")
lass.fit(df_train, outcome_train)
print("Accuracy of lasso regression on training set: ", 
      lass.score(df_train, outcome_train))

Accuracy of lasso regression on training set:  1.0


In [38]:
y_pred = lass.predict(df_test)
print("Accuracy of lasso regression on test set: ", 
      ((outcome_test == y_pred).sum()) / df_test.shape[0])

Accuracy of lasso regression on test set:  0.907928388746803


### Report

The feature engineering I did before running these models included creating four different features that focused on the year the house was built. Additionally, I created a feature about whether or not the house was "affordable" ($165000 or less), and this became the outcome I was looking to predict based on the other features in the dataset. 

When evaluating the models, I focused on the r-squared score and the difference between the r-squared score for the training set versus the test set. 

The strongest model of the three was the lasso logistic regression model. This model had the highest accuracy and the smallest difference between the test and train r-squared scores. 

There are many strengths of regression as a modeling approach because the different features allow the model to build different algorithms that create the line that best fits the data.